<a href="https://colab.research.google.com/github/taianecbassis/pollutant-salvador-ba/blob/main/data-analysis/ssa_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Importing required python packages*** -

In [ ]:
from platform import python_version
print ('Python version used in this Jupyter Nootbook:', python_version())

Python version used in this Jupyter Nootbook: 3.10.12


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).




***Data preview and general check***



In [ ]:
dfssa_cab = pd.read_excel('/content/drive/MyDrive/Mestrado/Dados/DADOS BRUTOS/dados_SSA_CAB.xlsx')


In [ ]:
dfssa_cab.head()

,Data_CAB,Hora_CAB,SO2_CAB,CO_CAB,O3_CAB,MP_CAB,NO_CAB,NOX_CAB,NO2_CAB,VEL_VENTO_CAB,DIR_VENTO_CAB,TEMP_CAB,UMID_CAB,STWD_CAB,Unnamed: 14
0,1/1/2011,1,0.4,0.48,13.6,26.3,114.61,122.19,7.59,3.4,135,25.7,83,13.1,NaN
1,1/1/2011,2,0.5,0.57,13.7,23.6,122.64,131.69,9.05,3.8,131,25.6,82,13.9,NaN
2,1/1/2011,3,0.5,0.64,12.5,23.6,123.62,134.72,11.09,3.1,128,25.4,83,11.6,NaN
3,1/1/2011,4,0.6,0.72,8.2,22.9,154.94,164.36,9.42,1.5,127,25.1,86,20.9,NaN
4,1/1/2011,5,0.4,0.69,6.6,23,169.24,177.21,7.99,1.3,119,25,87,19.2,NaN


In [ ]:
dfssa_cab.tail()

,Data_CAB,Hora_CAB,SO2_CAB,CO_CAB,O3_CAB,MP_CAB,NO_CAB,NOX_CAB,NO2_CAB,VEL_VENTO_CAB,DIR_VENTO_CAB,TEMP_CAB,UMID_CAB,STWD_CAB,Unnamed: 14
52603,31/12/2016,20,0.3,0.35,7,25.7,25.15,42.15,17.17,2.9,104,26.8,72,19.5,0
52604,31/12/2016,21,0.2,0.33,7.2,19.7,25.23,41.01,15.78,2.8,101,26.6,73,19.3,0
52605,31/12/2016,22,0.2,0.38,7.1,19.6,31.56,50.29,18.76,2.6,104,26.7,75,19.6,0
52606,31/12/2016,23,0.3,0.34,7.7,11.5,20.06,35.86,16.05,2.9,101,26.4,76,17.9,0
52607,31/12/2016,24,0.5,0.32,8.2,7.6,13.1,26.05,13.54,2.9,96,26,78,16.9,0




***Data preprocessing***

In [ ]:
#Converting the Data_CAB column into the datetime type and Hora_CAB column into the timedelta type
dfssa_cab['Data_CAB'] = pd.to_datetime(dfssa_cab['Data_CAB'], format = '%d/%m/%Y')
dfssa_cab['Hora_CAB'] = pd.to_timedelta(dfssa_cab['Hora_CAB'], unit= 'h')

In [ ]:
print(dfssa_cab['Data_CAB'])
print()
print(dfssa_cab['Hora_CAB'])

0       2011-01-01
1       2011-01-01
2       2011-01-01
3       2011-01-01
4       2011-01-01
           ...    
52603   2016-12-31
52604   2016-12-31
52605   2016-12-31
52606   2016-12-31
52607   2016-12-31
Name: Data_CAB, Length: 52608, dtype: datetime64[ns]

0       0 days 01:00:00
1       0 days 02:00:00
2       0 days 03:00:00
3       0 days 04:00:00
4       0 days 05:00:00
              ...      
52603   0 days 20:00:00
52604   0 days 21:00:00
52605   0 days 22:00:00
52606   0 days 23:00:00
52607   1 days 00:00:00
Name: Hora_CAB, Length: 52608, dtype: timedelta64[ns]


In [ ]:
#Concatenate the Data_CAB and Hora_CAB columns into the date_time column
#Converting the dataframe into a time series with the date_time column as index and defining the frequency type as hourly
dfssa_cab['Date_time'] = dfssa_cab['Data_CAB'] + dfssa_cab['Hora_CAB']
#dfssa_cab_ts = dfssa_cab.set_index('date_time')['MP_CAB']
dfssa_cab = dfssa_cab.set_index('Date_time')
dfssa_cab = dfssa_cab.asfreq(freq='H')
dfssa_cab = dfssa_cab.drop(['Data_CAB','Hora_CAB'], axis=1)
dfssa_cab = dfssa_cab.drop(dfssa_cab.columns[12], axis=1)
print(dfssa_cab.head())

                    SO2_CAB CO_CAB O3_CAB MP_CAB  NO_CAB NOX_CAB NO2_CAB  \
Date_time                                                                  
2011-01-01 01:00:00     0.4   0.48   13.6   26.3  114.61  122.19    7.59   
2011-01-01 02:00:00     0.5   0.57   13.7   23.6  122.64  131.69    9.05   
2011-01-01 03:00:00     0.5   0.64   12.5   23.6  123.62  134.72   11.09   
2011-01-01 04:00:00     0.6   0.72    8.2   22.9  154.94  164.36    9.42   
2011-01-01 05:00:00     0.4   0.69    6.6     23  169.24  177.21    7.99   

                    VEL_VENTO_CAB DIR_VENTO_CAB TEMP_CAB UMID_CAB STWD_CAB  
Date_time                                                                   
2011-01-01 01:00:00           3.4           135     25.7       83     13.1  
2011-01-01 02:00:00           3.8           131     25.6       82     13.9  
2011-01-01 03:00:00           3.1           128     25.4       83     11.6  
2011-01-01 04:00:00           1.5           127     25.1       86     20.9  
2011-

In [ ]:
MP_10 = dfssa_cab['MP_CAB'].loc['2011-01-01 01:00:00':'2011-02-01 00:00:00']
print(dfssa_cab['MP_CAB'])
#MP_10.plot(figsize=(12, 6))
#plt.title('Variação das concentrações de MP10 - Janeiro de 2001')
#plt.xlabel('Data e Hora')
#plt.ylabel('Concentração MP10')
#plt.show()


Date_time
2011-01-01 01:00:00    26.3
2011-01-01 02:00:00    23.6
2011-01-01 03:00:00    23.6
2011-01-01 04:00:00    22.9
2011-01-01 05:00:00      23
                       ... 
2016-12-31 20:00:00    25.7
2016-12-31 21:00:00    19.7
2016-12-31 22:00:00    19.6
2016-12-31 23:00:00    11.5
2017-01-01 00:00:00     7.6
Freq: H, Name: MP_CAB, Length: 52608, dtype: object
